In [1]:
import awkward as ak
import hist
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema
from coffea.analysis_tools import PackedSelection
import json
import numpy as np
import mplhep as hep
import matplotlib.pyplot as plt

In [2]:
filename = "/home/prayag/Coffea/data/3BF5BC57-5D24-1C4D-B277-7B0BAA5DF3C7.root"
events = NanoEventsFactory.from_root(
    filename,
    schemaclass=NanoAODSchema.v7
).events()

In [9]:
mode = "Z2Jets_NuNu_Zpt"
if mode.startswith("MET_Run2018"):
    key = "MET_Run2018"
elif "Jets_NuNu" in mode :
    key = "ZJets_NuNu"
key
        

'ZJets_NuNu'